# Notes
https://access.redhat.com/solutions/4795571

In [11]:
# imports
import base64
import json
import subprocess
import tempfile
from pathlib import Path
from pprint import pprint
from shutil import make_archive, rmtree
from urllib.parse import quote, quote_plus
from uuid import uuid4

import requests


In [12]:
# settings

namespace = "redhat-rhoam-observability"
query = "ALERT{}"

In [13]:
# getting secret
# secret = subprocess.run(["oc", "get", "secret", "-n", namespace, "|", "grep",  "prometheus-k8s-token", "|", "head", "-n1", "|", "awk", "'{ print $1 }'"])
data = subprocess.run(["oc", "get", "secret", "-n", namespace], capture_output=True)
data = data.stdout.decode()
data = data.split("\n")
secret = None
for d in data:
    a = d.split(" ")
    if a[0].startswith("prometheus-k8s-token"):
        secret = a[0]
        break


print(secret)

prometheus-k8s-token-92skh


In [14]:
# getting token
data1 = subprocess.run(["oc", "get", "secret", secret, "-n", namespace, "-o", "jsonpath='{.data.token}" ], capture_output=True)
data1 = data1.stdout.decode()
token = base64.b64decode(data1).decode()

print(token)


eyJhbGciOiJSUzI1NiIsImtpZCI6Ik10S2ZWQ0NkWWcxbElKZEVRNF9ESXpYdGluVW9yc09jM3pqNDR3Q3E0X0kifQ.eyJpc3MiOiJrdWJlcm5ldGVzL3NlcnZpY2VhY2NvdW50Iiwia3ViZXJuZXRlcy5pby9zZXJ2aWNlYWNjb3VudC9uYW1lc3BhY2UiOiJyZWRoYXQtcmhvYW0tb2JzZXJ2YWJpbGl0eSIsImt1YmVybmV0ZXMuaW8vc2VydmljZWFjY291bnQvc2VjcmV0Lm5hbWUiOiJwcm9tZXRoZXVzLWs4cy10b2tlbi05MnNraCIsImt1YmVybmV0ZXMuaW8vc2VydmljZWFjY291bnQvc2VydmljZS1hY2NvdW50Lm5hbWUiOiJwcm9tZXRoZXVzLWs4cyIsImt1YmVybmV0ZXMuaW8vc2VydmljZWFjY291bnQvc2VydmljZS1hY2NvdW50LnVpZCI6IjVhOTczYTkyLWExNDYtNDU0ZC1hNTk1LWM3ODRjZGZjODQxNiIsInN1YiI6InN5c3RlbTpzZXJ2aWNlYWNjb3VudDpyZWRoYXQtcmhvYW0tb2JzZXJ2YWJpbGl0eTpwcm9tZXRoZXVzLWs4cyJ9.XhJocX9zjPWz0byu6AwAHTqWPWSP0btA6HVCd3NlEHbkau-dA1xUufXRx3tSsm5ecVoHLfIL0Ke_z3gtPYK6YY0Vzxblc4VL5BzShqs6ag89n7OBfv78i4d_X5050-9S3ndrY1nTnmsrcmZnm16X57P5iyP3wKxiWFPjD0tThoXuUGx1RTSI8Ugzc_AGFksQ8UgXRApRLMnil5xTg9c3j-BJD4AnO5gn4FD_q0qheKr14BnjIKTJg83G9XvyJOh1v5DG5ejTWemB09RwdtbMOKtseNSbklqcOtWBcGX5-r4U_QS_XaFkzLuOaIPAmVj_J52Zj-GATYelEvc77aOhHA6qpeta5ZBJjYA9iM8WZtKb

In [15]:
# getting prometheus host
data2 = subprocess.run(["oc", "get", "route", "prometheus", "-n", namespace, "-o", "jsonpath={.spec.host}" ], capture_output=True)
host = data2.stdout.decode()
print(host)

prometheus-redhat-rhoam-observability.apps.jfitzpat.uq32.s1.devshift.org


In [18]:
# query the api

# get container memory
query = "max by(container) (container_memory_usage_bytes{namespace=~'.+rhoam.+'})"
query = quote_plus(query)
resp = requests.get(
    f'https://{host}/api/v1/query?query={query}',
    headers={"Authorization": f"Bearer {token}"}
)



In [20]:
print(resp)


<Response [403]>
